In [0]:
from databricks.sdk.service.serving import ChatMessage
from databricks.sdk import WorkspaceClient
import pandas as pd

In [0]:
w = WorkspaceClient()

In [0]:
def generate_sql_statement(text, tablename):
    column_names = []
    colNameDF = spark.sql(f"select * from {tablename}").limit(0)
    column_namesStr = ", ".join(colNameDF.columns)
    # Change it to your own input
    messages = [
        {
        "role": "user",
        "content": "Hello!"
        },
        {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
        },
        {
        "role": "user",
        "content": f"Please construct sql query, in one line, using table {tablename} from statement: {text} given the columns: {column_namesStr}"
        }
    ]
    messages = [ChatMessage.from_dict(message) for message in messages]
    response = w.serving_endpoints.query(
        name="databricks-dbrx-instruct",
        messages=messages,
    )
    return response.as_dict()

In [0]:
response = generate_sql_statement("return distinct regions where price is less than 4000", "workspace.default.vehicles_small_100")
print(response)

{'choices': [{'index': 0, 'message': {'content': '```sql\nSELECT DISTINCT region FROM workspace.default.vehicles_small_100 WHERE price < 4000;\n```', 'role': 'assistant'}}], 'created': 1718052464, 'id': 'chatcmpl_e1030483-70c9-4fbb-991c-11e67ea36816', 'model': 'dbrx-instruct-032724', 'object': 'chat.completion', 'usage': {'completion_tokens': 22, 'prompt_tokens': 338, 'total_tokens': 360}}


In [0]:
def process_response(response):
    msg = response["choices"][0]["message"]["content"]
    sql = ""
    if "SELECT" in msg:
        sql = msg[msg.find("SELECT"):]
        sql = sql.replace(';', "")
        sql = sql.replace('\n', "")
        sql = sql.replace('`', "")
    return sql

In [0]:
sql = process_response(response)

In [0]:
def describe_output(key_val_pairs):
    messages = [
        {
        "role": "user",
        "content": "Hello!"
        },
        {
        "role": "assistant",
        "content": "Hello! How can I assist you today?"
        },
        {
        "role": "user",
        "content": f"Please convert the following key value pairs: {key_val_pairs} into a descriptive form in single quotes"
        }
    ]
    messages = [ChatMessage.from_dict(message) for message in messages]
    response = w.serving_endpoints.query(
        name="databricks-dbrx-instruct",
        messages=messages,
    )
    response_msg = response.as_dict()
    output_msg = response_msg["choices"][0]["message"]["content"]
    #print(output_msg)
    #output_msg = output_msg[output_msg.rfind(":"):].strip()
    output_msg = output_msg.split("'")[1]
    output_msg = "The following is the top found matching result: " + output_msg
    return output_msg

In [0]:
def generate_explanation_from_sql(sql):
    df = spark.sql(sql).toPandas()
    df = df.head(1)
    top_record = df.to_dict(orient='records')
    #print(top_record)
    return describe_output(top_record)


In [0]:
def ansqwer_query(text):
    #tablename = "workspace.default.vehicles_small_100"
    tablename = "workspace.default.vehicles"
    response = generate_sql_statement(text, tablename)
    sql = process_response(response).lower()
    #print(sql)
    return generate_explanation_from_sql(sql)

In [0]:
ansqwer_query("give ids and makes for the the honda with price 5000 and above")

'The following is the top found matching result: Manufacturer: Honda, Model: Civic LX Sedan, ID: 7316067403'

In [0]:
%sql
SELECT * FROM workspace.default.vehicles WHERE manufacturer = 'tes' AND price >= 5000

id,url,region,region_url,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,image_url,description,county,state,lat,long,posting_date
